In [6]:
from scrapy import Selector

import requests

import pandas as pd

from tqdm.notebook import tqdm

tqdm.pandas()

In [7]:
# Loads dataframe which contains all games
all_games = pd.read_csv('../data/raw data/all_games.csv')

In [8]:
# Remove the ', [, and ] characters from the 'link' column in 'all_games.csv' so that scrapy can access the links
all_games['link'] = all_games['link'].str.replace('[', '')
all_games['link'] = all_games['link'].str.replace(']', '')
all_games['link'] = all_games['link'].str.replace("'", '')


In [9]:
# Function to scrape the genre information of a game
def get_genre_info(link):
    response = requests.get(link)

    sel = Selector(text = response.text)

    containers = sel.css('div.info-genres')

    titles = containers.css('dl dt::text').getall()

    values = containers.css('dl dd')

    values = [values.css('a::text').getall() for values in values]

    genre = dict(zip(titles, values))

    return pd.Series(genre).to_frame().T

In [10]:
# Applies the function to all the games in the dataframe
genre_series = all_games['link'].progress_apply(get_genre_info)

  0%|          | 0/1017 [00:00<?, ?it/s]

In [11]:
# Turn the series into a dataframe
genre_info = pd.concat(genre_series.tolist(), ignore_index = True)


In [12]:
# Add titles to the dataframe so we have a common key in each dataframe
genre_info['title'] = all_games['title']

In [13]:
# Save the dataframe to a csv file
genre_info.to_csv('../data/raw data/genre_info.csv', index = False)